<a href="https://colab.research.google.com/github/UW-CTRL/AA548-spr23/blob/main/homework/hw_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AA/EE/ME 548: Homework 04

## Name:

### Export page instructions:
Make sure all the text and figures are shown in your PDF print out. Try zooming out / widening your browser window etc when you print to PDF. Whatever needs to be done so that all the relevant content is shown in the PDF.

### Homework feedback:

**What was the coolest thing(s) you learned from lectures/homework that you didn't know previously?**


**Which problem did you find the most challenging and why?**


**Approximate how long did you spend on this homework?**


**Feedback on this homework (can improve for future hw and years)**


## Submission details
To submit your homework, please save a PDF copy of your notebook (make sure all the cells are run correctly, and the correct outputs are displayed), and submit via Canvas.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import cvxpy as cp
plt.style.use('seaborn-ticks')


<ipython-input-1-fd7eff59c6d0>:6: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-ticks')


## Problem 1

You may find the following code template useful. But you are not required to use it if you / feel free to edit it however you like / write your own version of the code





In [2]:
# set up trajectory optimization problem but with initial state as a parameter that can be updated
def setup_trajopt(A_dyn, B_dyn, Q, R, Qt, goal, u_max, MPC_horizon):
    n = A_dyn.shape[0]
    m = B_dyn.shape[1]
    us = cp.Variable([MPC_horizon,m], name="controls")
    xs = cp.Variable([MPC_horizon+1,n], name="states")
    # initial state is a parameter. It can be updated before solving the problem
    initial_state = cp.Parameter(n, name="initial_state")
    
    objective = ... # fill code
    constraints = ... # fill code

    problem = cp.Problem(cp.Minimize(objective), constraints)
    return problem

# function to update the value of initial_state
def update_initial_state(problem, initial_state):
    problem.param_dict["initial_state"].project_and_assign(initial_state)
    return problem




In [3]:
dt = 0.1    # time step size
n = 4   # state dim
m = 2   # control dim

Q = np.diag([5., 5., 1., 1.]) 
Qt = np.eye(n) * 8.0
R = np.diag([1., 1.])
goal_state = np.zeros(n)
initial_state = np.array([-4, 0., 0., 2.])
u_max = 2.  # maximum control magnitude
N_MPC_horizon = 10
N_total_horizon = 50
Sigma_vec = np.array([0.0, 0.0, .1, .1])
np.random.seed(seed=123)
noise_vec = np.random.randn(N_MPC_horizon, n) * Sigma_vec

A_dynamics = np.array([[1., 0., dt, 0.],
                        [0., 1., 0., dt],
                        [0., 0., 1., 0.],
                        [0., 0., 0., 1.]])

B_dynamics = np.array([[0.5 * dt**2, 0.],
                        [0., 0.5 * dt**2],
                        [dt, 0.],
                        [0., dt]])  

def simulate_system(initial_state, controls, noise_vec):
    '''
    initial_state has size [n,]
    controls has size [T,m]
    noise_vec has size [T,n]
    '''

    T = controls.shape[0]
    n = initial_state.shape[0]
    states = np.zeros([T+1, n])
    states[0] = initial_state
    for (t,ctrl) in enumerate(controls):
        states[t+1] = A_dynamics @ states[t] + B_dynamics @ ctrl + noise_vec[t]
    return states

## (a)

In [ ]:
MPC_states = ...  # size [time_horizon+1, n]
MPC_controls = ...  # size [time_horizon, m]
true_states = ...  # size [time_horizon+1, n]


In [ ]:
plt.figure(figsize=(15,5))

plt.subplot(1,2,1)
plt.plot(MPC_states[:,0], MPC_states[:,1], label="MPC trajectory")
plt.quiver(MPC_states[:,0], MPC_states[:,1], MPC_states[:,2], MPC_states[:,3], scale=30, width=.003)
plt.quiver(MPC_states[:-1,0], MPC_states[:-1,1], MPC_controls[:,0], MPC_controls[:,1], scale=30, width=.003, color='red')

plt.plot(true_states[:,0], true_states[:,1], label="true trajectory")
plt.quiver(true_states[:,0], true_states[:,1], true_states[:,2], true_states[:,3], scale=30, width=.003)
plt.quiver(true_states[:-1,0], true_states[:-1,1], MPC_controls[:,0], MPC_controls[:,1], scale=30, width=.003, color='red')
plt.title("Optimal Trajectory with velocity and acceleration vector")
plt.legend()
plt.xlabel("$x$ position")
plt.ylabel("$y$ position")
plt.grid()

plt.subplot(1,2,2)
plt.plot(MPC_controls)
plt.plot(np.linalg.norm(MPC_controls, 2, axis=1), '--')
plt.title("Optimal Control Sequence")
plt.xlabel("Time (s)")
plt.ylabel("Acceleration")
plt.legend(["$a_x$", "$a_y$", "$\|a\|_2$"])
plt.grid()

## (b)

In [ ]:
# pre-defining the noise applied to the system
np.random.seed(seed=123) # play around with different seeds
noise_vec = np.random.randn(N_total_horizon, n) * np.array([0.0, 0.0, .1, .1])

In [ ]:
# your MPC code

MPC_states = ...  # size [time_horizon+1, n]
MPC_controls = ...  # size [time_horizon, n]
OL_states = ...  # size [time_horizon+1, n]
OL_controls = ...  # size [time_horizon, n]

In [ ]:
plt.figure(figsize=(15,5))

plt.subplot(1,2,1)
plt.plot(MPC_states[:,0], MPC_states[:,1], label="MPC trajectory", marker='.')
plt.plot(OL_states[:,0], OL_states[:,1], label="true OL trajectory", marker='.')

plt.title("Optimal Trajectory with velocity and acceleration vector")
plt.legend()
plt.xlabel("$x$ position")
plt.ylabel("$y$ position")
plt.grid()

plt.subplot(1,2,2)
plt.plot(MPC_controls)
plt.plot(np.linalg.norm(MPC_controls, 2, axis=1), '--')

plt.plot(OL_controls)
plt.plot(np.linalg.norm(OL_controls, 2, axis=1), '--')

plt.title("Optimal Control Sequence")
plt.xlabel("Time (s)")
plt.ylabel("Acceleration")
plt.legend(["MPC $a_x$", "MPC $a_y$", "MPC $\|a\|_2$", "OL $a_x$", "OL $a_y$", "OL $\|a\|_2$"])
plt.grid()

# Problem 2 Kalman Filter

## (a)

## (b)

## (c)

## (d)

In [4]:
# your code here. Make sure you label your plot clearly
# Recommend using plt.plot and plt.scatter to clearly show the state at each time step

## (e)

## (f)

In [ ]:
# you may find these plotting functionalities useful. Or you can use your own code

def create_95_ellipse_patch(mean, covariance):
    chi_95 = 5.991
    def eigsorted(cov):
        vals, vecs = np.linalg.eigh(cov)
        order = vals.argsort()[::-1]
        return vals[order], vecs[:,order]
    vals, vecs = eigsorted(covariance)
    theta = np.degrees(np.arctan2(*vecs[:,0][::-1]))
    w, h = 2  * np.sqrt(vals * chi_95)
    return Ellipse(xy=mean, width=w, height=h, angle=theta)


def plot_95_ellipse(mean, covariance):
    chi_95 = 5.991
    mean = np.array([[mean[0]],[mean[1]]])
    val, rot = np.linalg.eig(covariance)
    val = np.sqrt(val * chi_95)
    t = np.linspace(0, 2.0 * np.pi, 1000)
    xy = np.stack((np.cos(t), np.sin(t)), axis=-1)

    plt.plot(*(rot @ (val * xy).T + mean))
    plt.fill(*(rot @ (val * xy).T + mean), alpha=0.3)

In [5]:
# your code

## (g)

In [ ]:
# your code

## (h)

In [ ]:
# your code 